<a href="https://colab.research.google.com/github/Sandeeppasala/EVA_S5/blob/master/EVA4_Session_F5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(20)
        self.conv2 = nn.Conv2d(20, 16, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(16)
        self.conv4 = nn.Conv2d(16, 10, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(10)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(10, 10, 3)
        self.bn5 = nn.BatchNorm2d(10)
        self.conv6 = nn.Conv2d(10, 10, 3)
        self.bn6 = nn.BatchNorm2d(10)
        self.conv7 = nn.Conv2d(10, 10, 3)
        self.dout = nn.Dropout(p=0.1)
        self.gap = nn.AdaptiveAvgPool2d((1,1))   

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        #x = self.dout(x)
        x = self.pool1(F.relu(self.bn2(self.conv2(x))))
        x = self.dout(x)
        x = F.relu(self.bn3(self.conv3(x)))
        #x = self.dout(x)
        x = self.pool2(F.relu(self.bn4(self.conv4(x))))
        x = self.dout(x)
        x = F.relu(self.bn5(self.conv5(x)))
        #x = self.dout(x)
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.conv7(x)
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [0]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 20, 28, 28]             200
       BatchNorm2d-2           [-1, 20, 28, 28]              40
            Conv2d-3           [-1, 16, 28, 28]           2,896
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         MaxPool2d-5           [-1, 16, 14, 14]               0
           Dropout-6           [-1, 16, 14, 14]               0
            Conv2d-7           [-1, 16, 14, 14]           2,320
       BatchNorm2d-8           [-1, 16, 14, 14]              32
            Conv2d-9           [-1, 10, 14, 14]           1,450
      BatchNorm2d-10           [-1, 10, 14, 14]              20
        MaxPool2d-11             [-1, 10, 7, 7]               0
          Dropout-12             [-1, 10, 7, 7]               0
           Conv2d-13             [-1, 10, 5, 5]             910
      BatchNorm2d-14             [-1, 1

In [0]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)



Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [0]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Loss=0.06940914690494537 Batch_id=468 Accuracy=92.24: 100%|██████████| 469/469 [00:11<00:00, 40.16it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0567, Accuracy: 9830/10000 (98.30%)



Loss=0.05881337448954582 Batch_id=468 Accuracy=97.95: 100%|██████████| 469/469 [00:11<00:00, 40.22it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0447, Accuracy: 9874/10000 (98.74%)



Loss=0.037995561957359314 Batch_id=468 Accuracy=98.30: 100%|██████████| 469/469 [00:11<00:00, 41.34it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0226, Accuracy: 9930/10000 (99.30%)



Loss=0.008815889246761799 Batch_id=468 Accuracy=99.15: 100%|██████████| 469/469 [00:11<00:00, 40.77it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0209, Accuracy: 9925/10000 (99.25%)



Loss=0.003155246376991272 Batch_id=468 Accuracy=99.22: 100%|██████████| 469/469 [00:11<00:00, 41.37it/s]



Test set: Average loss: 0.0209, Accuracy: 9931/10000 (99.31%)



Target:

As the parameters are reduced add batch normalization to the model.

Results:

Parameters: 9K

Best Training Accuracy: 99.22%

Best Test Accuracy: 99.31%

Analysis:

Model is light weight as params are reduced

This is a fit model. 

Model can be further improved.


